In [ ]:
%pip install tqdm

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from models.commit import Commit
from models.file import File
from typing import List

In [ ]:
def process_files(files: List['File'], com: Commit):
    for file in files:
        file_data = File(file, com.repo_name, com.org_name, file.split('.')[-1].lower())
        File.add_file(file_data)

In [ ]:
commits = Commit.fetch_all_commits()
parent_folder = path.join('..', 'download', 'orgs')

In [ ]:
# def exec_task(com: 'Commit'):
#     files = Commit.get_file_names_from_git(path.join(parent_folder, com.org_name, com.repo_name), com.sha)
#     if not files:
#         return
#     process_commit(files, com)

# with ThreadPoolExecutor(max_workers=15) as executor:
#     futures = {executor.submit(exec_task, com): com for com in tqdm(commits, total=len(commits))}
    
#     for future in tqdm(as_completed(futures), total=len(futures)):
#         future.result()

for com in commits:
    files = Commit.get_file_names_from_git(path.join(parent_folder, com.org_name, com.repo_name), com.sha)
    if not files:
        continue
    process_files(files, com)